In [6]:
import cv2
import numpy as np
import pandas as pd

In [7]:
def measure_pixel_resolution(img_path, known_length_mm, pixels_across):
    """Estimate pixel resolution (mm/pixel) for a given image."""
    pixel_res = known_length_mm / pixels_across
    print(f"{img_path}: Pixel resolution = {pixel_res:.3f} mm/pixel")
    return pixel_res

def compute_dof(focal_length_mm, f_number, subject_distance_mm, coc_mm):
    """Compute depth of field (DOF) using optical formula."""
    dof_near = (subject_distance_mm * (subject_distance_mm - focal_length_mm)) / \
               (subject_distance_mm + (focal_length_mm / f_number) * (subject_distance_mm / coc_mm) - focal_length_mm)
    dof_far = (subject_distance_mm * (subject_distance_mm - focal_length_mm)) / \
              (subject_distance_mm - (focal_length_mm / f_number) * (subject_distance_mm / coc_mm) - focal_length_mm)
    dof = dof_far - dof_near if np.isfinite(dof_far) else np.inf
    return dof, dof_near, dof_far

In [8]:
images_info = [
    {"file": "35mm.png", "focal_length": 35, "pixels_across": 60},
    {"file": "50mm.png", "focal_length": 50, "pixels_across": 50},
    {"file": "85mm.png", "focal_length": 85, "pixels_across": 45}
]

known_length_mm = 100   # Known real-world object size
f_number = 5.6          # Aperture (can change)
subject_distance_mm = 1000  # Distance to subject

In [9]:
results = []

for info in images_info:
    img_path = info["file"]
    focal_length = info["focal_length"]
    pixels_across = info["pixels_across"]

    # Load image (optional visualization)
    try:
        img = cv2.imread(img_path)
        if img is not None:
            print(f"Loaded {img_path}, size: {img.shape[1]}x{img.shape[0]}")
        else:
            print(f"Warning: Could not load {img_path}, using default resolution assumption.")
    except Exception as e:
        print(f"Error loading {img_path}: {e}")

    # Compute pixel resolution
    pixel_res = measure_pixel_resolution(img_path, known_length_mm, pixels_across)

    # Approximate CoC (circle of confusion)
    coc_mm = 2 * pixel_res

    # Compute DOF
    dof, dof_near, dof_far = compute_dof(focal_length, f_number, subject_distance_mm, coc_mm)

    results.append({
        "Image": img_path,
        "Focal Length (mm)": focal_length,
        "CoC (mm)": coc_mm,
        "DOF Near (mm)": dof_near,
        "DOF Far (mm)": dof_far,
        "Total DOF (mm)": dof
    })


Loaded 35mm.png, size: 1172x776
35mm.png: Pixel resolution = 1.667 mm/pixel
Loaded 50mm.png, size: 1174x781
50mm.png: Pixel resolution = 2.000 mm/pixel
Loaded 85mm.png, size: 1178x776
85mm.png: Pixel resolution = 2.222 mm/pixel


In [10]:

df = pd.DataFrame(results)
print("\n=== Depth of Field Comparison ===")
print(df.to_string(index=False))


=== Depth of Field Comparison ===
   Image  Focal Length (mm)  CoC (mm)  DOF Near (mm)  DOF Far (mm)  Total DOF (mm)
35mm.png                 35  3.333333     339.788732  -1060.439560    -1400.228293
50mm.png                 50  4.000000     298.540965   -740.947075    -1039.488040
85mm.png                 85  4.444444     211.307683   -365.973859     -577.281542
